# Map Me If You Can
- Tested with Jupyter version > 5.0 && < 6.0
- Tutorials see [ArcGIS API for Python](https://developers.arcgis.com/python/guide/)

In [1]:
!pip install --user arcgis

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [76]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
import pandas

## Connect to ArcGIS Online

In [4]:
geospatial = GIS()

## Show me a map centered on Bonn

In [100]:
def get_world_map():
    return geospatial.map()

def get_bonn_map():
    return geospatial.map("Bonn")

bonn_map = get_bonn_map()
bonn_map

MapView(layout=Layout(height='400px', width='100%'))

## Use the CORONAVIRUS tracker API
- Only world countries or county level for the US

In [92]:
import urllib.request

def extract_from_json(data_frame, parent, child=None):
    if None is child:
        return data_frame.apply(lambda record: record[0][parent], axis=1)
    else:
        return data_frame.apply(lambda record: record[0][parent][child], axis=1)

def tracker_locations_to_dataframe():
    url = "https://coronavirus-tracker-api.herokuapp.com/v2/locations"
    with urllib.request.urlopen(url) as stream:
        locations_complex = pandas.read_json(stream)
        locations = pandas.DataFrame()
        locations["latitude"] = extract_from_json(locations_complex, "coordinates", "latitude").astype(float)
        locations["longitude"] = extract_from_json(locations_complex, "coordinates", "longitude").astype(float)
        locations["country"] = extract_from_json(locations_complex, "country")
        locations["country_code"] = extract_from_json(locations_complex, "country_code")
        locations["last_updated"] = extract_from_json(locations_complex, "last_updated")
        locations["last_updated"] = locations.apply(lambda record: pandas.to_datetime(record["last_updated"]), axis=1)
        locations["confirmed"] = extract_from_json(locations_complex, "latest", "confirmed").astype(int)
        locations["deaths"] = extract_from_json(locations_complex, "latest", "deaths").astype(int)
        locations["recovered"] = extract_from_json(locations_complex, "latest", "recovered").astype(int)
        return locations
    
def tracker_locations_by_country_code(country_code):
    locations = tracker_locations_to_dataframe()
    return locations.loc[locations["country_code"] == country_code]

def tracker_locations_to_spatial():
    locations = tracker_locations_to_dataframe()
    return GeoAccessor.from_xy(locations, x_column="longitude", y_column="latitude", sr=4326)

In [101]:
tracker_locations = tracker_locations_to_spatial()
world_map = get_world_map()
tracker_locations.spatial.plot(map_widget=world_map)
world_map

MapView(layout=Layout(height='400px', width='100%'))

## Use the Robert Koch Institute table for German cases
- Query the table statistics from ArcGIS Online
- Aggregate the table statistics

In [118]:
from arcgis.features import Table

def get_case_sum():
    corona_feature_table = Table("https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0")
    result_set = corona_feature_table.query(where="1=1", out_statistics=[{"statisticType": "sum", "onStatisticField": "AnzahlFall"}])
    rows = result_set.features
    if 1 == len(rows):
        return result_set.features[0].attributes["SUM_AnzahlFall"]
    else:
        return 0

def get_case_sum_by_county():
    corona_feature_table = Table("https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0")
    result_set = corona_feature_table.query(where="1=1", group_by_fields_for_statistics="Bundesland", out_statistics=[{"statisticType": "sum", "onStatisticField": "AnzahlFall"}])
    rows = result_set.features
    if 0 < len(rows):
        return result_set.sdf
    else:
        return None
    
def get_case_sum_by_county_and_ages():
    corona_feature_table = Table("https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0")
    result_set = corona_feature_table.query(where="1=1", group_by_fields_for_statistics="Altersgruppe,Bundesland", out_statistics=[{"statisticType": "sum", "onStatisticField": "AnzahlFall"}])
    rows = result_set.features
    if 0 < len(rows):
        return result_set.sdf
    else:
        return None

get_case_sum_by_county_and_ages()

,SUM_AnzahlFall,Altersgruppe,Bundesland
0,30,A00-A04,Baden-Württemberg
1,84,A05-A14,Baden-Württemberg
2,958,A15-A34,Baden-Württemberg
3,1862,A35-A59,Baden-Württemberg
4,599,A60-A79,Baden-Württemberg
...,...,...,...
101,5,A05-A14,Thüringen
102,50,A15-A34,Thüringen
103,107,A35-A59,Thüringen
104,23,A60-A79,Thüringen
